In [15]:
import re
registro = ""
palavras = []
with open("../corpus/Bosque_CF_7.4_cgd.txt", "r", encoding = "ISO-8859-1") as file:
    for line in file:
        
        if len(registro) > 0:
            palavras.append(line)
        
        if line.find("id=\"2") >= 0:
            registro = line
        
        if len(registro) > 0 and line.find("</s>") >= 0:
            break
            
registro

textRegex = re.compile(r'text=\"([^\"]+)\"')
frase = textRegex.findall(registro)[0]
frase

'BRASÍLIA Pesquisa Datafolha publicada hoje revela um dado supreendente: recusando uma postura radical, a esmagadora maioria (77%) dos eleitores quer o PT participando do Governo Fernando Henrique Cardoso.'

In [16]:
import ufal.udpipe
# ufal.udpipe.Model etc. are SWIG-magic and cannot be detected by pylint
# pylint: disable=no-member

class Model:
    def __init__(self, path):
        """Load given model."""
        self.model = ufal.udpipe.Model.load(path)
        if not self.model:
            raise Exception("Cannot load UDPipe model from file '%s'" % path)

    def tokenize(self, text):
        """Tokenize the text and return list of ufal.udpipe.Sentence-s."""
        tokenizer = self.model.newTokenizer(self.model.DEFAULT)
        if not tokenizer:
            raise Exception("The model does not have a tokenizer")
        return self._read(text, tokenizer)

    def read(self, text, in_format):
        """Load text in the given format (conllu|horizontal|vertical) and return list of ufal.udpipe.Sentence-s."""
        input_format = ufal.udpipe.InputFormat.newInputFormat(in_format)
        if not input_format:
            raise Exception("Cannot create input format '%s'" % in_format)
        return self._read(text, input_format)

    def _read(self, text, input_format):
        input_format.setText(text)
        error = ufal.udpipe.ProcessingError()
        sentences = []

        sentence = ufal.udpipe.Sentence()
        while input_format.nextSentence(sentence, error):
            sentences.append(sentence)
            sentence = ufal.udpipe.Sentence()
        if error.occurred():
            raise Exception(error.message)

        return sentences

    def tag(self, sentence):
        """Tag the given ufal.udpipe.Sentence (inplace)."""
        self.model.tag(sentence, self.model.DEFAULT)

    def parse(self, sentence):
        """Parse the given ufal.udpipe.Sentence (inplace)."""
        self.model.parse(sentence, self.model.DEFAULT)

    def write(self, sentences, out_format):
        """Write given ufal.udpipe.Sentence-s in the required format (conllu|horizontal|vertical)."""

        output_format = ufal.udpipe.OutputFormat.newOutputFormat(out_format)
        output = ''
        for sentence in sentences:
            output += output_format.writeSentence(sentence)
        output += output_format.finishDocument()

        return output

In [48]:
model = Model('portuguese-ud-1.2-160523.udpipe')
sentences = model.tokenize(frase)

In [50]:
for s in sentences:
    model.tag(s)
    model.parse(s)
model.write(sentences, "conllu")

'# newdoc\n# newpar\n# sent_id = 1\n# text = BRASÍLIA Pesquisa Datafolha publicada hoje revela um dado supreendente: recusando uma postura radical, a esmagadora maioria (77%) dos eleitores quer o PT participando do Governo Fernando Henrique Cardoso.\n1\tBRASÍLIA\tBrasília\tADJ\tadj|F|S\tGender=Fem|Number=Sing\t2\tamod\t_\t_\n2\tPesquisa\tpesquisa\tNOUN\tn|F|S\tGender=Fem|Number=Sing\t6\tnsubj\t_\t_\n3\tDatafolha\tDatafolha\tPROPN\tprop|F|S\tGender=Fem|Number=Sing\t2\tnmod\t_\t_\n4\tpublicada\tpublicar\tVERB\tv-pcp|F|S\tGender=Fem|Number=Sing|VerbForm=Part\t2\tacl\t_\t_\n5\thoje\thoje\tADV\tadv\t_\t4\tadvmod\t_\t_\n6\trevela\trevelar\tVERB\tv-fin|PR|3S|IND\tMood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin\t0\troot\t_\t_\n7\tum\tum\tDET\tart|<arti>|M|S\tDefinite=Ind|Gender=Masc|Number=Sing|PronType=Art\t8\tdet\t_\t_\n8\tdado\tdado\tNOUN\tn|M|S\tGender=Masc|Number=Sing\t6\tdobj\t_\t_\n9\tsupreendente\tsurpreendente\tADJ\tadj|<ALT>|M|S\tGender=Masc|Number=Sing|Typo=Yes\t8\tamod\t_\tSp